# Track files on local and cloud storage

```{note}

Within a Jupyter notebook, the call to `ln.nb.header()` tracks the notebook run as a data source.

Learn more: {doc}`/guide/run`.
```

In [ ]:
import lamindb as ln

ln.nb.header()

## Track local files

In [ ]:
filepath = ln.dev.datasets.file_jpg_paradisi05()
filepath

We'll work with a single class for data objects in memory and on disk: {class}`~lamindb.DObject`. On disk, these are often (but not always, e.g., for `zarr`) files. Instantiating `DObject` creates a `dobject` record:

In [ ]:
dobject = ln.DObject(filepath)

The `dobject` record captures metadata about the file and will be our way to query and load data.

In [ ]:
dobject

We can also access linked metadata records, for instance, the record that stores metadata about this run.

In [ ]:
dobject.source

As we're ingesting from a notebook, here, it defaults to the notebook run you saw printed above.

In [ ]:
assert ln.nb.run == dobject.source

If we want to add metadata & data to database & storage, we can do so in a single transaction:

In [ ]:
ln.add(dobject)

Getting the data back works through `.load()` - here, we get back a cryptic filename. More on this below.

In [ ]:
dobject.load()

You can also query the data object. One of the simplest ways is by name:

In [ ]:
ln.select(ln.DObject, name="paradisi05_laminopathic_nuclei").one()

Learn more: {doc}`/guide/select`.

## What's in the DB now?

In [ ]:
ln.view()

## And what's in storage?

Two cryptically named files:

In [ ]:
!ls mydata

If you prefer semantic names, you can easily achieve it by tracking existing data rather than ingesting data into a storage location: {doc}`/faq/existing-data`.

Naming data objects in storage by the primary key ID of the `DObject` is typically preferred when facing potential clashes of names at large scale or working with in-memory views.